### MLP with Tensorboard

see:  https://github.com/nlintz/TensorFlow-Tutorials/blob/master/9_tensorboard.py


In [2]:
#!/usr/bin/env python

import tensorflow as tf
import input_data


def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01), name=name)

# This network is the same as the previous one except with an extra hidden layer + dropout
def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden):
    # Add layer name scopes for better graph visualization
    with tf.name_scope("layer1"):
        X = tf.nn.dropout(X, p_keep_input)
        h = tf.nn.relu(tf.matmul(X, w_h))
    with tf.name_scope("layer2"):
        h = tf.nn.dropout(h, p_keep_hidden)
        h2 = tf.nn.relu(tf.matmul(h, w_h2))
    with tf.name_scope("layer3"):
        h2 = tf.nn.dropout(h2, p_keep_hidden)
        return tf.matmul(h2, w_o)

mnist = input_data.read_data_sets(".", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

X = tf.placeholder("float", [None, 784], name="X")
Y = tf.placeholder("float", [None, 10], name="Y")

w_h = init_weights([784, 625], "w_h")
w_h2 = init_weights([625, 625], "w_h2")
w_o = init_weights([625, 10], "w_o")

# Add histogram summaries for weights
tf.histogram_summary("w_h_summ", w_h)
tf.histogram_summary("w_h2_summ", w_h2)
tf.histogram_summary("w_o_summ", w_o)

p_keep_input = tf.placeholder("float", name="p_keep_input")
p_keep_hidden = tf.placeholder("float", name="p_keep_hidden")
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
    train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
    # Add scalar summary for cost
    tf.scalar_summary("cost", cost)

with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(py_x, 1)) # Count correct predictions
    acc_op = tf.reduce_mean(tf.cast(correct_pred, "float")) # Cast boolean to float to average
    # Add scalar summary for accuracy
    tf.scalar_summary("accuracy", acc_op)

with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter("./logs/nn_logs", sess.graph) # for 0.8
    merged = tf.merge_all_summaries()

    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})
        summary, acc = sess.run([merged, acc_op], feed_dict={X: teX, Y: teY,
                                          p_keep_input: 1.0, p_keep_hidden: 1.0})
        writer.add_summary(summary, i)  # Write summary
        print(i, acc)                   # Report the accuracy

(0, 0.93989998)
(1, 0.96850002)
(2, 0.9702)
(3, 0.97589999)
(4, 0.97589999)
(5, 0.97719997)
(6, 0.97689998)
(7, 0.97790003)
(8, 0.9788)
(9, 0.97970003)
(10, 0.9799)
(11, 0.98000002)
(12, 0.98009998)
(13, 0.9813)
(14, 0.98019999)
(15, 0.9795)
(16, 0.98269999)
(17, 0.98159999)
(18, 0.98089999)
(19, 0.98229998)
(20, 0.98159999)
(21, 0.9824)
(22, 0.9824)
(23, 0.98250002)
(24, 0.98290002)
(25, 0.98229998)
(26, 0.98259997)
(27, 0.98180002)
(28, 0.98269999)
(29, 0.98360002)
(30, 0.98210001)
(31, 0.98229998)
(32, 0.98229998)
(33, 0.9824)
(34, 0.98110002)
(35, 0.9817)
(36, 0.98199999)
(37, 0.98229998)
(38, 0.98369998)
(39, 0.98479998)
(40, 0.98009998)
(41, 0.98089999)
(42, 0.98280001)
(43, 0.98040003)
(44, 0.98229998)
(45, 0.98110002)
(46, 0.98150003)
(47, 0.98229998)
(48, 0.9824)
(49, 0.9799)
(50, 0.98320001)
(51, 0.98220003)
(52, 0.98299998)
(53, 0.98320001)
(54, 0.98379999)
(55, 0.98189998)
(56, 0.98360002)
(57, 0.98379999)
(58, 0.98409998)
(59, 0.98369998)
(60, 0.98449999)
(61, 0.98220003)


/Users/charlesmartin14/anaconda2/lib/python2.7/gzip.py:275: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
input_data.py:35: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)
